In [58]:
import pandas as pd 
import numpy as np
import random 
import sqlite3 
from sqlalchemy import create_engine
from scipy.stats import norm
from sklearn.linear_model import LinearRegression
import cPickle

In [6]:
patient_cnx = sqlite3.connect('patient.db')
austin_db = pd.read_sql_query("SELECT * FROM austin_patients", patient_cnx )

In [21]:
#Add age to each person. 
possible_ages = range(18, 100)

def add_age(x): 
    x = random.choice(possible_ages)
    return x

In [30]:
austin_db['age'] = pd.Series(np.random.randint(low=8, high=100, size= len(austin_db.ix[:, 1])), index=austin_db.index)

In [36]:
#Add height for each person.
women_height_distribution = norm.rvs(loc=63.708774, scale=2.696284, size=700, random_state=42)
male_height_distribution = norm.rvs(loc=69.026346, scale=2.863362, size=700, random_state=42)

In [67]:
#Add weight for each person. 
women_weight_distribution = norm.rvs(loc=135.860093, scale=19.022468, size=700, random_state=42)
male_weight_distribution = norm.rvs(loc=187.020621, scale=19.781155, size=700, random_state=42)

In [68]:
austin_db['weight'] = austin_db.apply(lambda row: random.choice(male_weight_distribution) if row['gender'] == 'M' else random.choice(women_weight_distribution), axis=1)

In [71]:
austin_db.head(20)

,index,id,gender,firstname,lastname,street,city,region,country,zipcode,phone,age,height,weight
0,0,1,M,Burnu,Hope,1778 Red Centre,Austin,TX,US,78741,(930) 646-5885,90,66.756064,197.173644
1,1,2,M,Burnu,Hope,1778 Red Centre,Austin,TX,US,78725,(930) 646-5885,10,65.997737,164.411348
2,2,3,F,Sudevi,Hamm,5054 Rocky View Route,Austin,TX,US,78723,(325) 784-6765,28,66.746313,113.965355
3,3,4,F,Eryn,Heal,6922 Golden Falls,Austin,TX,US,78610,(281) 591-6577,72,68.703042,142.651409
4,4,5,F,Clorinda,Kirkpatrick,6183 Round Way,Austin,TX,US,78751,(936) 841-6234,13,65.902227,141.205252
5,5,6,M,Hadden,McMinn,4514 Lazy Pony Avenue,Austin,TX,US,78729,(956) 598-8612,62,65.993434,193.431376
6,6,7,F,Aloha,Beer,5088 Cinder Quail Crescent,Austin,TX,US,78737,(972) 677-3087,37,62.626970,184.811748
7,7,8,F,Katarina,Pepper,8177 Fallen Rabbit Round,Austin,TX,US,78732,(903) 221-1154,38,63.112977,143.165966
8,8,9,F,Estelle,Grubbs,3665 Broad Terrace,Austin,TX,US,78742,(940) 727-6418,78,60.417033,139.347710
9,9,10,M,Singh,Winfrey,9738 Blue Quay,Austin,TX,US,78703,(930) 809-8791,91,67.087387,184.732964


In [75]:
#Now that the faux database is done. It is time to put it into a PostgresSQL Database. 
engine = create_engine('postgresql://earlynreinhardt@localhost:5432/athena_database')
austin_db.to_sql('atx_patient_db', engine)